<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=fd724551d0a3b74240ff0e1757762af5e370bf150b927f99ba12617dabebfdc7
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-30 09:28:44
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: -15.57 K (-0.11%)
Current PnL: -18.11 L (-11.87%)
CY Booked + Current PnL: -5.98 L (-3.92%)
-------------------
Total profit:  2.76 L
Total loss:  -20.87 L
-------------------
Total Booked + Current PnL: 21.00 L (16.72%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.3%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.18 L (58.31%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.99%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-6.91,72.0,X-LC,4.58,233438.0,23072.0,5719.0,-0.52,10.97,2.45,13.68,37.0,4.03,1.61,29.30,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,108.11,60.0,M-SC,1.29,89663.0,-11114.0,11208.0,-1.73,-11.03,12.50,0.09,245.0,-0.99,0.62,17.80,OX40N,NTT,DURABLES
12,BANKINDIA,149.0,-15.53,75.0,H-MC,17.02,216704.0,36896.0,12460.0,0.09,20.52,5.75,27.45,88.0,2.96,1.50,61.60,XR,NTT,BANKS
2,ABBOTINDIA,35195.0,-9.64,41.0,X-MC,2.50,89175.0,-1503.0,16408.0,0.07,-1.66,18.40,16.44,101.0,-0.09,0.62,18.18,X40,ATH,PHARMA
42,ITC,452.0,-36.66,61.0,X-LC,1.28,204549.0,4411.0,16487.0,-0.78,2.20,8.06,10.44,4.0,0.27,1.41,7.89,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,78.51,76.0,H-SC,2.71,154824.0,-27951.0,139016.0,6.55,-15.29,89.79,60.77,125.0,-0.20,1.07,38.67,XR,NTT,JEWELLERY
17,CAMS,5211.76,-3.39,55.0,X-SC,0.51,109897.0,7893.0,36035.0,1.80,7.74,32.79,43.06,122.0,0.22,0.76,26.18,X40N,ATH,MISC
10,BAJAJHFL,181.50,-13.00,55.0,X-MC,5.84,183319.0,-17187.0,114336.0,1.02,-8.57,62.37,48.45,90.0,-0.15,1.27,5.71,X40N,ATH,FINANCE
71,SURYODAY,240.00,73.25,65.0,H-SC,15.59,158865.0,-20206.0,85930.0,0.72,-11.28,54.09,36.71,135.0,-0.24,1.10,57.63,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-36.88,67.0,M-SC,3.68,130750.0,15952.0,78084.0,0.71,13.90,59.72,81.92,223.0,0.20,0.90,54.79,XR,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.00,15.14,48.0,H-SC,12.93,256686.0,-24437.0,157297.0,-1.79,-8.69,61.28,47.26,144.0,-0.16,1.77,37.53,XY24,NTT,INSURANCE
79,TTKPRESTIG,770.00,108.11,60.0,M-SC,1.29,89663.0,-11114.0,11208.0,-1.73,-11.03,12.50,0.09,245.0,-0.99,0.62,17.80,OX40N,NTT,DURABLES
62,SAIL,228.00,51.56,63.0,M-MC,10.31,239289.0,14327.0,153336.0,-1.13,6.37,64.08,74.53,192.0,0.09,1.65,40.68,XY24,BTT,STEEL
37,INDIAMART,4810.62,-53.33,56.0,H-SC,6.67,130528.0,7192.0,124432.0,-0.84,5.83,95.33,106.72,119.0,0.06,0.90,30.57,AR,ATH,MISC
32,HINDZINC,730.22,27.88,49.0,M-LC,11.34,208670.0,3594.0,108967.0,-0.81,1.75,52.22,54.89,52.0,0.03,1.44,26.27,X5K,ATH,METALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.0,-29.46,64.0,H-SC,15.88,201392.0,1438.0,95782.0,-0.17,0.72,47.56,48.63,115.0,0.02,1.39,12.36,XR,NTT,MISC
87,WIPRO,420.0,-14.25,47.0,M-LC,5.65,150747.0,-198.0,109653.0,0.36,-0.13,72.74,72.51,53.0,-0.00,1.04,5.88,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,108.11,60.0,M-SC,1.29,89663.0,-11114.0,11208.0,-1.73,-11.03,12.50,0.09,245.0,-0.99,0.62,17.80,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,53.0,H-SC,2.85,226278.0,-43389.0,79718.0,-0.50,-16.09,35.23,13.47,138.0,-0.54,1.56,15.06,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-21.86,56.0,H-MC,6.40,105650.0,-26185.0,68408.0,0.40,-19.86,64.75,32.03,98.0,-0.38,0.73,18.34,OX40N,NTT,IT
18,CERA,9475.0,-23.39,48.0,H-SC,2.52,139794.0,-36109.0,78131.0,-0.79,-20.53,55.89,23.89,149.0,-0.46,0.97,20.73,OX40N,NTT,CERAMICS
68,SIS,528.0,2031.09,45.0,H-SC,3.20,86012.0,-25020.0,48631.0,-0.81,-22.53,56.54,21.26,156.0,-0.51,0.59,16.21,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.0,-15.53,75.0,H-MC,17.02,216704.0,36896.0,12460.0,0.09,20.52,5.75,27.45,88.0,2.96,1.5,61.6,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,51.56,63.0,M-MC,10.31,239289.0,14327.0,153336.0,-1.13,6.37,64.08,74.53,192.0,0.09,1.65,40.68,XY24,BTT,STEEL
57,RECLTD,446.00,50.38,60.0,M-LC,5.16,210540.0,7370.0,34760.0,-0.71,3.63,16.51,20.74,55.0,0.21,1.46,10.74,XY25,NTT,FINANCE
32,HINDZINC,730.22,27.88,49.0,M-LC,11.34,208670.0,3594.0,108967.0,-0.81,1.75,52.22,54.89,52.0,0.03,1.44,26.27,X5K,ATH,METALS
47,KPIGREEN,731.05,12.48,74.0,H-SC,5.48,134077.0,8780.0,50145.0,0.07,7.01,37.40,47.03,141.0,0.18,0.93,67.88,MH,ATH,POWER
80,UNIONBANK,163.00,-8.53,64.0,M-LC,9.71,166116.0,25276.0,19220.0,-0.01,17.95,11.57,31.59,66.0,1.32,1.15,49.19,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-15.53,75.0,H-MC,17.02,216704.0,36896.0,12460.0,0.09,20.52,5.75,27.45,88.0,2.96,1.50,61.60,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-36.88,67.0,M-SC,3.68,130750.0,15952.0,78084.0,0.71,13.90,59.72,81.92,223.0,0.20,0.90,54.79,XR,NTT,DURABLES
47,KPIGREEN,731.05,12.48,74.0,H-SC,5.48,134077.0,8780.0,50145.0,0.07,7.01,37.40,47.03,141.0,0.18,0.93,67.88,MH,ATH,POWER
37,INDIAMART,4810.62,-53.33,56.0,H-SC,6.67,130528.0,7192.0,124432.0,-0.84,5.83,95.33,106.72,119.0,0.06,0.90,30.57,AR,ATH,MISC
32,HINDZINC,730.22,27.88,49.0,M-LC,11.34,208670.0,3594.0,108967.0,-0.81,1.75,52.22,54.89,52.0,0.03,1.44,26.27,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,20.40,149320.0,-127203.0,237269.0,-0.04,-46.00,158.90,39.81,54.0,-0.54,1.03,5.24,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.44,28.0,X-SC,4.95,89026.0,-39644.0,84940.0,-0.41,-30.81,95.41,35.20,219.0,-0.47,0.62,2.18,X40,NTT,FOOTWEAR
24,DMART,5391.45,-16.99,37.0,X-LC,9.88,101506.0,-1080.0,27894.0,0.20,-1.05,27.48,26.13,38.0,-0.04,0.70,24.25,X40N,ATH,FMCG
53,PGHH,17907.65,-30.55,38.0,X-MC,2.29,201135.0,315.0,67481.0,-0.33,0.16,33.55,33.76,80.0,0.00,1.39,5.36,X40,ATH,FMCG
31,HINDUNILVR,2922.00,-11.65,39.0,X-LC,7.53,248080.0,-8983.0,44109.0,-0.29,-3.49,17.78,13.67,24.0,-0.20,1.72,16.20,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-2.18,49.0,H-LC,1.17,157150.0,-28945.0,76422.0,-0.18,-15.55,48.63,25.51,15.0,-0.38,1.09,15.81,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,61.0,X-LC,1.28,204549.0,4411.0,16487.0,-0.78,2.20,8.06,10.44,4.0,0.27,1.41,7.89,X40,NTT,FMCG
53,PGHH,17907.65,-30.55,38.0,X-MC,2.29,201135.0,315.0,67481.0,-0.33,0.16,33.55,33.76,80.0,0.00,1.39,5.36,X40,ATH,FMCG
29,HAVELLS,2069.16,-5.33,50.0,X-MC,3.00,316071.0,-11920.0,118463.0,-0.28,-3.63,37.48,32.48,92.0,-0.10,2.19,7.38,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,0.74,51.0,X-MC,4.19,211470.0,19728.0,18038.0,-0.63,10.29,8.53,19.69,99.0,1.09,1.46,17.79,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.44,28.0,X-SC,4.95,89026.0,-39644.0,84940.0,-0.41,-30.81,95.41,35.20,219.0,-0.47,0.62,2.18,X40,NTT,FOOTWEAR
52,PAGEIND,51769.93,-27.18,42.0,X-MC,6.66,82240.0,-1270.0,21300.0,-0.58,-1.52,25.90,23.98,82.0,-0.06,0.57,4.21,X40,ATH,APPARELS
21,COLPAL,3767.14,-3.45,49.0,X-MC,7.66,225370.0,-37995.0,151336.0,-0.62,-14.43,67.15,43.04,84.0,-0.25,1.56,4.63,XY25,ATH,FMCG
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,20.40,149320.0,-127203.0,237269.0,-0.04,-46.00,158.90,39.81,54.0,-0.54,1.03,5.24,XY24,NTT,AUTO
53,PGHH,17907.65,-30.55,38.0,X-MC,2.29,201135.0,315.0,67481.0,-0.33,0.16,33.55,33.76,80.0,0.00,1.39,5.36,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-44.35,55.0,X-SC,37.88,55199.0,-9807.0,160734.0,0.30,-15.09,291.19,232.18,198.0,-0.06,0.38,7.67,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.18,42.0,X-MC,6.66,82240.0,-1270.0,21300.0,-0.58,-1.52,25.90,23.98,82.0,-0.06,0.57,4.21,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.49,45.0,X-SC,4.77,82457.0,-62703.0,138635.0,-0.50,-43.20,168.13,52.31,136.0,-0.45,0.57,10.37,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.64,41.0,X-MC,2.50,89175.0,-1503.0,16408.0,0.07,-1.66,18.40,16.44,101.0,-0.09,0.62,18.18,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.44,28.0,X-SC,4.95,89026.0,-39644.0,84940.0,-0.41,-30.81,95.41,35.20,219.0,-0.47,0.62,2.18,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-28.04,54.0,X-LC,10.94,287179.0,-58777.0,125469.0,-0.08,-16.99,43.69,19.28,1.0,-0.47,1.99,6.17,X40,ATH,IT
40,INFY,2275.00,-17.80,57.0,X-LC,5.77,319820.0,6932.0,164771.0,-0.59,2.22,51.52,54.87,3.0,0.04,2.21,10.68,X40,BTT,IT
42,ITC,452.00,-36.66,61.0,X-LC,1.28,204549.0,4411.0,16487.0,-0.78,2.20,8.06,10.44,4.0,0.27,1.41,7.89,X40,NTT,FMCG
84,VBL,671.64,-12.06,69.0,X-LC,8.37,316448.0,606.0,112054.0,0.11,0.19,35.41,35.67,5.0,0.01,2.19,14.07,X40N,ATH,FMCG
1,ABB,7934.00,-39.67,54.0,H-LC,5.61,252216.0,-9403.0,128605.0,-0.65,-3.59,50.99,45.57,7.0,-0.07,1.74,7.32,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7083.33,61.0,L-SC,7.89,78807.0,-15003.0,91621.0,0.43,-15.99,116.26,81.67,269.0,-0.16,0.54,52.61,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,28.16,80928.0,-32621.0,72673.0,-0.35,-28.73,89.80,35.27,268.0,-0.45,0.56,98.07,XR,NTT,HOTELS
79,TTKPRESTIG,770.00,108.11,60.0,M-SC,1.29,89663.0,-11114.0,11208.0,-1.73,-11.03,12.50,0.09,245.0,-0.99,0.62,17.80,OX40N,NTT,DURABLES
50,MASFIN,398.61,-19.60,50.0,H-SC,9.15,91590.0,-6390.0,27990.0,0.63,-6.52,30.56,22.05,152.0,-0.23,0.63,33.13,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-36.88,67.0,M-SC,3.68,130750.0,15952.0,78084.0,0.71,13.90,59.72,81.92,223.0,0.20,0.90,54.79,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,78.51,76.0,H-SC,2.71,154824.0,-27951.0,139016.0,6.55,-15.29,89.79,60.77,125.0,-0.20,1.07,38.67,XR,NTT,JEWELLERY
47,KPIGREEN,731.05,12.48,74.0,H-SC,5.48,134077.0,8780.0,50145.0,0.07,7.01,37.40,47.03,141.0,0.18,0.93,67.88,MH,ATH,POWER
86,WHIRLPOOL,2270.00,-36.88,67.0,M-SC,3.68,130750.0,15952.0,78084.0,0.71,13.90,59.72,81.92,223.0,0.20,0.90,54.79,XR,NTT,DURABLES
5,ANGELONE,3033.00,16.33,64.0,X-SC,5.55,205902.0,14896.0,42807.0,-0.03,7.80,20.79,30.21,157.0,0.35,1.42,30.31,X40N,NTT,FINANCE
69,SONACOMS,806.63,-31.93,71.0,M-SC,7.81,84752.0,-16407.0,56411.0,0.24,-16.22,66.56,39.54,202.0,-0.29,0.59,19.51,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.62
1,25,43.93
2,50,74.88


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.08
LC    32.28
MC    24.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.00
X40      16.19
XY25     12.00
XR       11.51
X40N     10.72
AR        7.83
OX40N     6.51
X200      1.76
X5K       1.44
SR        1.08
MH        0.93
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.47
X-LC    21.62
X-MC    18.76
M-SC    12.65
M-LC     5.09
H-LC     4.59
X-SC     4.53
H-MC     3.88
M-MC     1.65
L-SC     1.43
L-LC     0.98
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.54,-2.46,35.90
IT,12.47,-16.56,79.07
FINANCE,12.13,-7.68,56.50
MISC,6.84,-15.93,80.17
ELECTRICAL,5.80,-7.88,46.13
BANKS,5.67,-4.42,58.41
PAINTS,5.48,-16.43,33.78
INSURANCE,3.75,-2.07,40.13
AUTO,2.76,-44.07,104.20


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3230742.0,22
XR,1219418.0,14
AR,1156789.0,9
X40,817009.0,12
X40N,610201.0,9
OX40N,542411.0,9
XY25,447399.0,8
SR,265413.0,2
X5K,108967.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3290244.0,24
M-SC,1215364.0,15
X-MC,1167842.0,14
X-LC,1009512.0,13
X-SC,526042.0,6
H-LC,274607.0,3
M-LC,272600.0,4
H-MC,255941.0,3
L-SC,253318.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1152911.0      6
           AR         829095.0      5
           XR         751399.0      7
M-SC       XY24       697086.0      6
X-MC       XY24       570047.0      4
X-LC       X40        448286.0      5
           XY24       302335.0      2
X-MC       X40        283783.0      6
X-SC       X40N       280368.0      4
H-SC       SR         265413.0      2
           OX40N      241281.0      3
H-LC       AR         205027.0      2
X-LC       X40N       185195.0      3
H-MC       XY24       175073.0      1
X-MC       XY25       169374.0      2
L-SC       XR         164294.0      2
X-SC       XY24       160734.0      1
M-MC       XY24       153336.0      1
X-MC       X40N       144638.0      2
M-SC       OX40N      143698.0      4
           XY25       126897.0      1
           XR         125016.0      2
           AR         122667.0      2
M-LC       XR         109653.0      1
           X5K        108967.0      1
L-SC       OX40N       89024.0      1
X-SC       X40         84940.0      1
X-LC       XY25        73696.0      3
H-LC       X200        69580.0      1
H-MC       OX40N       68408.0      1
L-MC       XR          56596.0      1
H-SC       MH          50145.0      1
L-LC       XY25        42672.0      1
M-LC       XY25        34760.0      1
           XY24        19220.0      1
H-MC       XR          12460.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
